# Topic Modelling using LDA

#### Impoting Libraries

In [4]:
import pandas as pd
import numpy as np
import re
import os
#import langid # language identification (i.e. what language is this?)
import nltk.stem
import gensim
import string
import pyodbc

from nltk.corpus import stopwords 
from nltk.tag import PerceptronTagger
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction import text 

import gensim.corpora as corpora
from gensim.models import phrases
from gensim.models.phrases import Phraser
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim import corpora, models, similarities
import pyLDAvis.gensim

from itertools import chain
#import dill
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
os.chdir("C:\\Users\\300067285\\Desktop\\Surabhi\\E-Clerks")

In [6]:
twcs = pd.read_csv("twcs.csv")

In [7]:
len(twcs)

2811774

In [9]:
twcs.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [170]:
train_ds = twcs

In [171]:
train_ds['NOTE_TXT']=  twcs['text']

In [133]:
len(train_ds)

2811774

In [222]:
my_stop_words = text.ENGLISH_STOP_WORDS
new_words= set(('http','https','ll','hey','hello','hi','team','ve',
            're','ba','gt','day','time','week','month','date','amp','am','say','dm','sure','wkjhdxwgrq','gdrquypt'))
my_stop_words = my_stop_words.union(new_words)
len(my_stop_words)

339

In [172]:
train_ds['Campaign_tags'] = train_ds['NOTE_TXT'].apply(lambda x: re.findall(r'#\w+',x))

In [173]:
train_ds = train_ds[train_ds['Campaign_tags'].astype(str)  != '[]']

In [174]:
len(train_ds)
train_ds.dtypes

159126

tweet_id                     int64
author_id                   object
inbound                       bool
created_at                  object
text                        object
response_tweet_id           object
in_response_to_tweet_id    float64
NOTE_TXT                    object
Campaign_tags               object
dtype: object

In [176]:
train_ds = train_ds[train_ds.inbound == True]

In [177]:

len(train_ds)

143671

In [178]:
# data cleaning : removing @ and digits
train_ds['NOTE_TXT'] = train_ds['NOTE_TXT'].apply(lambda x: re.sub(r'(@\w*)|(\d*)','',x))


In [181]:
lemma = WordNetLemmatizer()
english_stemmer = nltk.stem.SnowballStemmer('english')
tagger = PerceptronTagger()
pos_tag = tagger.tag
exclude_puncts = set(string.punctuation) 

In [182]:
# POS Tagger
def get_wordnet_pos(treebank_token):
        w = treebank_token[0]
        treebank_tag = treebank_token[1]
        if treebank_tag.startswith('J'):
            return w,wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return w,wordnet.VERB
        elif treebank_tag.startswith('N'):
            return w,wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return w,wordnet.ADV
        else:
            # As default pos in lemmatization is Noun
            return w,wordnet.NOUN

In [183]:
def pre_tokenize(doc):
    tokenized_doc = " ".join([re.sub(' +',' ',re.sub('[^A-Za-z0-9 ]+', ' ',i)) for i in doc.lower().split()])
    puncexcluded_doc= ''.join(ch for ch in tokenized_doc if ch not in exclude_puncts)
    return puncexcluded_doc.split()

In [184]:
def lemmatized(doc):
    return lemma.lemmatize(*doc)

In [185]:
pre_tokenized = train_ds.NOTE_TXT.apply(pre_tokenize)

In [186]:
pre_tagged_tokens = list(map(pos_tag, pre_tokenized))

In [187]:
wordnet_tagged_tokens = [list(map(get_wordnet_pos, tagged_token)) for tagged_token in pre_tagged_tokens]

In [188]:
lemmatized_tokens = [list(map(lemmatized, doc)) for doc in wordnet_tagged_tokens]

In [189]:
import pickle

with open('Lemmatized tokens', 'wb') as fp:
    pickle.dump(lemmatized_tokens, fp)

In [190]:
train_ds['lemmatized'] = lemmatized_tokens

C:\Users\300067285\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [191]:
train_ds['lemmatized'][1:7].values

array([list(['thank', 'you', 'for', 'resolve', 'my', 'issue', 'so', 'quickly', 'y', 'all', 'be', 'the', 'best', 'fanforlife']),
       list(['name', 'a', 'good', 'halloween', 'duo', 'speedwaybrick', 'uttranscript', 'http', 't', 'co', 'shdrmlct']),
       list(['excellent', 'halloweening', 'countcolin', 'halloween', 'food', 'http', 't', 'co', 'lqcsoohdj']),
       list(['quand', 'ton', 'ne', 'ressemble', 'pa', 'celui', 'que', 'tu', 'a', 'r', 'serv', 'amp', 'qu', 'en', 'd', 'tre', 'sale', 'et', 'en', 'sous', 'sol', 'tu', 'te', 'retrouves', 'dans', 'logement', 'http', 't', 'co', 'bceihetq']),
       list(['kinda', 'need', 'to', 'make', 'sure', 'i', 'have', 'cdn', 'in', 'my', 'bank', 'so', 'would', 'be', 'nice', 'to', 'know', 'i', 'dun', 'just', 'have', 'cash', 'lay', 'around', 'ya', 'know', 'reallifestruggles']),
       list(['you', 'll', 'be', 'even', 'happy', 'to', 'know', 'that', 'the', 'restriction', 'around', 'the', 'volcano', 'area', 'have', 'be', 'lift', 'enjoy', 'stay', 'at', 'hom

In [224]:
mystop = my_stop_words
exclude = set(string.punctuation)

In [225]:
def rem_1letter_words(doc):
     return [word for word in doc if len(word)>1]

In [226]:
filtered = list(map(rem_1letter_words,lemmatized_tokens))
#filtered[0:5]

In [227]:
def rem_stop_words(doc):
    return [i for i in doc if i not in my_stop_words]
stop_free = list(map(rem_stop_words,filtered))

In [228]:
train_ds['lemmatized'] = stop_free
train_ds.lemmatized[0:7].values

C:\Users\300067285\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array([list(['costume', 'boorito']),
       list(['thank', 'resolve', 'issue', 'quickly', 'best', 'fanforlife']),
       list(['good', 'halloween', 'duo', 'speedwaybrick', 'uttranscript', 'shdrmlct']),
       list(['excellent', 'halloweening', 'countcolin', 'halloween', 'food', 'lqcsoohdj']),
       list(['quand', 'ton', 'ne', 'ressemble', 'pa', 'celui', 'que', 'tu', 'serv', 'qu', 'en', 'tre', 'sale', 'et', 'en', 'sous', 'sol', 'tu', 'te', 'retrouves', 'dans', 'logement', 'bceihetq']),
       list(['kinda', 'need', 'make', 'cdn', 'bank', 'nice', 'know', 'dun', 'just', 'cash', 'lay', 'ya', 'know', 'reallifestruggles']),
       list(['happy', 'know', 'restriction', 'volcano', 'area', 'lift', 'enjoy', 'stay', 'home', 'pal', 'unlucky'])],
      dtype=object)

In [229]:
train_ds_grp_clean = train_ds[train_ds['lemmatized'].astype(str)  != '[]']
train_ds_grp_clean.lemmatized[0:5].values

array([list(['costume', 'boorito']),
       list(['thank', 'resolve', 'issue', 'quickly', 'best', 'fanforlife']),
       list(['good', 'halloween', 'duo', 'speedwaybrick', 'uttranscript', 'shdrmlct']),
       list(['excellent', 'halloweening', 'countcolin', 'halloween', 'food', 'lqcsoohdj']),
       list(['quand', 'ton', 'ne', 'ressemble', 'pa', 'celui', 'que', 'tu', 'serv', 'qu', 'en', 'tre', 'sale', 'et', 'en', 'sous', 'sol', 'tu', 'te', 'retrouves', 'dans', 'logement', 'bceihetq'])],
      dtype=object)

In [230]:
mydictionary = corpora.Dictionary(train_ds_grp_clean.lemmatized)

In [231]:
type(train_ds_grp_clean.lemmatized)

pandas.core.series.Series

#### Bigram detection

In [232]:
# Create a bigram detector
bigram_detector = gensim.models.phrases.Phrases(train_ds_grp_clean['lemmatized'], min_count=10, threshold=1.0)
#bigram_implementer
bigram_implementer = gensim.models.phrases.Phraser(bigram_detector)

In [233]:
def id_bigrams_usingImplementer(eachdoc):
    return " ".join(bigram_implementer[eachdoc])

In [234]:
doc_withbigrams = [bigram_implementer[eachdoc] for eachdoc in train_ds_grp_clean.lemmatized]

In [235]:
mybigramdictionary = corpora.Dictionary(doc_withbigrams)
print(mybigramdictionary)

Dictionary(147915 unique tokens: ['boorito', 'costume', 'best', 'fanforlife', 'quickly']...)


In [237]:
mybigramdictionary.filter_extremes(no_below=5, no_above=0.9999)

In [238]:
print(mybigramdictionary)

Dictionary(19413 unique tokens: ['boorito', 'costume', 'best', 'quickly', 'resolve_issue']...)


#### Printing the bigram output

In [265]:
#j=0
#for i in range(0,19413):
#    word = mybigramdictionary.get(i)
#    if "_" in word:
#        j=j+1
#        print(mybigramdictionary.get(i))

#print(j)

In [240]:
doc_term_matrix_bg = [mybigramdictionary.doc2bow(doc) for doc in doc_withbigrams]

#### LDA model creation

In [241]:
# Running and Trainign LDA model on the document term matrix.
ldamodel_ssym_alpha_multi_10_online = models.LdaMulticore(doc_term_matrix_bg, num_topics=10, id2word = mybigramdictionary, 
                           passes=20,iterations = 500, alpha='asymmetric',eta='auto',chunksize=10000,workers = 7, eval_every=1,batch=False,random_state = 918273645)
                                                        #, gamma_threshold = 0.003,per_word_topics=True,minimum_phi_value= 0.10,minimum_probability=0.10)

In [242]:
from gensim.models.coherencemodel import CoherenceModel

NewCoherence2 = CoherenceModel(model=ldamodel_ssym_alpha_multi_10_online, corpus=doc_term_matrix_bg, dictionary=mybigramdictionary, coherence='u_mass')

NewCoherence2.get_coherence()

-5.16084021031671

In [243]:
topic_words = []
for i in range(10):
    topic_words.append(ldamodel_ssym_alpha_multi_10_online.print_topic(i,topn=5))

In [266]:
#for i in range(10):
 #   print("Topic " + str(i))
  #  print("Topic Keywords:")
   # ldamodel_ssym_alpha_multi_10_online.show_topic(i,20)

In [245]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

#### LDA model Visualization

In [246]:
preparedData = pyLDAvis.gensim.prepare(ldamodel_ssym_alpha_multi_10_online, doc_term_matrix_bg, mybigramdictionary, sort_topics=False)

In [247]:
pyLDAvis.display(preparedData)

In [248]:
pyLDAvis.save_json(preparedData, 'lda_model'+'.json')
pyLDAvis.save_html(preparedData, 'lda_model'+'.html')

In [249]:
#Need to manually input
topic_ids = np.arange(10)

topic_names=['Flight','Mobile','misc','Cus_service/Negative','Uber/Foods','Iphone','Music/Spotify/Amazon','ecommerce/payment','thanks/Love','customer_service']
            

len(topic_names)

10

In [250]:
def gettopntopics(row,n=1):
    tops = sorted(row, key=lambda y:y[1], reverse = True)
    return tops[:n]

In [253]:
full_corpus = [mybigramdictionary.doc2bow(doc) for doc in train_ds_grp_clean.lemmatized]
full_corpus_fitted = [gettopntopics(prob,3) for prob in ldamodel_ssym_alpha_multi_10_online[full_corpus]]



len(full_corpus_fitted)

143599

In [254]:
train_ds_grp_clean['fitted'] = full_corpus_fitted
Final_df_fullcorpus = train_ds_grp_clean.copy()

train_ds_grp_clean[1:2]
Final_df_fullcorpus[1:2]

C:\Users\300067285\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,NOTE_TXT,Campaign_tags,lemmatized,fitted
77,155,115735,True,Tue Oct 31 22:00:46 +0000 2017,@ChipotleTweets Thank you @ChipotleTweets for ...,154,156.0,Thank you for resolving my issue so quickly!...,[#fanforlife],"[thank, resolve, issue, quickly, best, fanforl...","[(9, 0.84115344), (0, 0.034746982), (1, 0.0258..."


,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,NOTE_TXT,Campaign_tags,lemmatized,fitted
77,155,115735,True,Tue Oct 31 22:00:46 +0000 2017,@ChipotleTweets Thank you @ChipotleTweets for ...,154,156.0,Thank you for resolving my issue so quickly!...,[#fanforlife],"[thank, resolve, issue, quickly, best, fanforl...","[(9, 0.84115344), (0, 0.034746982), (1, 0.0258..."


In [255]:
Final_df_fullcorpus[['RC_1', 'RC_2','RC_3']] = pd.DataFrame(Final_df_fullcorpus['fitted'].values.tolist(), 
                                             index= Final_df_fullcorpus.index, columns=['RC_1', 'RC_2','RC_3'])



Final_df_fullcorpus[['RC1', 'RC1_prob']] = pd.DataFrame(Final_df_fullcorpus['RC_1'].values.tolist(), 
                                             index= Final_df_fullcorpus.index, columns=['RC1', 'RC1_prob'])

In [256]:
Final_df_fullcorpus['RC_2'].fillna('na',inplace=True)
Final_df_fullcorpus['RC_3'].fillna('na',inplace=True)

In [257]:
Final_df_fullcorpus[['RC2', 'RC2_prob']] = pd.DataFrame(Final_df_fullcorpus['RC_2'].values.tolist(), 
                                             index= Final_df_fullcorpus.index, columns=['RC2', 'RC_prob'])



Final_df_fullcorpus[['RC3', 'RC3_prob']] = pd.DataFrame(Final_df_fullcorpus['RC_3'].values.tolist(), 
                                             index= Final_df_fullcorpus.index, columns=['RC_3', 'RC3_prob'])

In [258]:
Final_df_fullcorpus.drop(columns = ['fitted','RC_1','RC_2','RC_3'],inplace = True)

topic_df_dict =  dict(zip(topic_ids, topic_names))

Final_df_fullcorpus['RC1'] = Final_df_fullcorpus.RC1.map(topic_df_dict)
Final_df_fullcorpus['RC2'] = Final_df_fullcorpus.RC2.map(topic_df_dict)
Final_df_fullcorpus['RC3'] = Final_df_fullcorpus.RC3.map(topic_df_dict)


In [259]:
Final_df_fullcorpus.to_csv("Final_df_fullcorpus.csv",index=False)